In [ ]:
# 한글 시작
from pyhwpx import Hwp
hwp = Hwp()

In [ ]:
# 스타일 적용하기
hwp.set_style(style_number) # 0~1

In [ ]:
# 위치 이동
hwp.Run("moveDocBegin")
hwp.Run("moveDocEnd")

# 블록 이동
hwp.Run("MoveSelNextParaBegin")

In [ ]:
# 메시지 박스 종류
#
# hwp.SetMessageBocMode(~) 방식으로 사용
#

hwp.SetMessageBoxMode(0x111111) # 모두 예
hwp.SetMessageBoxMode(0x222222) # 모두 아니오 
hwp.SetMessageBoxMode(0xFFFFFF) # 모두 초기화


#
#
#
#
# MB_MASK = 0x00FFFFFF
# 
# 1. 확인(MB_OK) : IDOK(1)
#    MB_OK_IDOK : 0x00000001
#    MB_OK_MASK : 0x0000000F
# 
# 2. 확인/취소(MB_OKCANCEL) : IDOK(1), IDCANCEL(2)
#    MB_OKCANCEL_IDOK : 0x00000010
#    MB_OKCANCEL_IDCANCEL : 0x00000020
#    MB_OKCANCEL_MASK : 0x00000030
# 
# 3. 종료/재시도/무시(MB_ABORTRETRYIGNORE) : IDABORT(3), IDRETRY(4), IDIGNORE(5)
#    MB_ABORTRETRYIGNORE_IDABORT : 0x00000100
#    MB_ABORTRETRYIGNORE_IDRETRY : 0x00000200
#    MB_ABORTRETRYIGNORE_IDIGNORE : 0x00000400
#    MB_ABORTRETRYIGNORE_MASK : 0x00000F00
# 
# 4. 예/아니오/취소(MB_YESNOCANCEL) : IDYES(6), IDNO(7), IDCANCEL(2)
#    MB_YESNOCANCEL_IDYES : 0x00001000
#    MB_YESNOCANCEL_IDNO : 0x00002000
#    MB_YESNOCANCEL_IDCANCEL : 0x00004000
#    MB_YESNOCANCEL_MASK : 0x0000F000
# 
# 5. 예/아니오(MB_YESNO) : IDYES(6), IDNO(7)
#    MB_YESNO_IDYES : 0x00010000
#    MB_YESNO_IDNO : 0x00020000
#    MB_YESNO_MASK : 0x000F0000 << 기본값 >>
# 
# 6. 재시도/취소(MB_RETRYCANCEL) : IDRETRY(4), IDCANCEL(2)
#    MB_RETRYCANCEL_IDRETRY : 0x00100000
#    MB_RETRYCANCEL_IDCANCEL : 0x00200000
#    MB_RETRYCANCEL_MASK : 0x00F00000

In [ ]:
# 찾기

# 문단번호가 있는지 찾기
i = 0
while hwp.set_pos(0, i, 0): 
    if hwp.GetHeadingString():
        prop = hwp.ParaShape
        
        
# 특정 스타일 찾기
def search_style(style_number):
	hwp.HAction.GetDefault("Goto", hwp.HParameterSet.HGotoE.HSet);
	hwp.HParameterSet.HGotoE.HSet.SetItem("DialogResult", style_number);
	hwp.HParameterSet.HGotoE.SetSelectionIndex = 4;
	hwp.HAction.Execute("Goto", hwp.HParameterSet.HGotoE.HSet);

'''
여기서 SetSelectionIndex는 검색 대상을 지정한다.
1: 쪽, 2: 책갈피, 3: 머리말, 4: 스타일, 5: 각주, 6: 미주, 7: 구역
'''




def extract_text_from_block():  
    hwp.InitScan(range=0xff)  # 0xff <<선택된 범위 내에서 검색
    _, text_blokced = hwp.GetText()  # 텍스트만 추출
    hwp.ReleaseScan() # 스캔을 해제.
    return text_blokced # 이경우, 해당 target이 text에 포함되어있으면 True, 아니면 False를 반환.




def search_text(text,direction="Forward"):
    """
    특정 텍스트를 찾아서 블록 처리
    Args:
        text (str): 찾을 텍스트
        direction (optional): 찾기 방향(Forward, Backward, AllDoc)
    """
    
    hwp.HAction.GetDefault("RepeatFind", hwp.HParameterSet.HFindReplace.HSet)
    hwp.HParameterSet.HFindReplace.FindString = text
    hwp.HParameterSet.HFindReplace.Direction = hwp.FindDir(f"{direction}")
    hwp.HParameterSet.HFindReplace.FindType = 1  # 일반 텍스트 찾기
    hwp.HParameterSet.HFindReplace.UseWildCards = 1
    hwp.HParameterSet.HFindReplace.IgnoreMessage = 2  # 계속 찾을까요? 1=YES, 2=NO
    hwp.HAction.Execute("RepeatFind", hwp.HParameterSet.HFindReplace.HSet)


'''
FindType 종류
0: 서식
1: 일반텍스트
2: 일반텍스트(정규식)
3: 특수문자
4: 필드
5: 스타일 찾기

* 정규식 사용 시 FindType = 1 & UseWildCards =1 로 해야함
* 조건식 사용시 Findtype

IgnoreMessage 종류
0: 모든 메시지 표시
1: "예(Yes)" 자동 선택
2: "아니오(No)" 자동 선택
3: 모든 메시지 무시
'''


def search_text_condition(text): 
    """조건식사용에 체크하고 텍스트 검색

	Args:
		text (_type_): _description_
	"""
    hwp.SetMessageBocMode(0x000000) # 모두 아니오
    hwp.HAction.GetDefault("RepeatFind", hwp.HParameterSet.HFindReplace.HSet)
    hwp.HParameterSet.HFindReplace.FindString = f"{text}"
    hwp.HParameterSet.HFindReplace.Direction = hwp.FindDir("Forward")
    hwp.HParameterSet.HFindReplace.FindType = 1
    hwp.HParameterSet.HFindReplace.FindRegExp = 1 
    hwp.HParameterSet.HFindReplace.IgnoreMessage = 2
    hwp.HAction.Execute("RepeatFind", hwp.HParameterSet.HFindReplace.HSet)
    hwp.SetMessageBocMode(0xFFFFFF) # 모두 아니오 
 
 
 # search_text_condition(r"{\d\d년}.\d월}|{\d\d년.\d\d월}")
 # 조건식 사용시 FindType = 1 & FindRegExp = 1 로 설정하면 됨.
 


def go_to_index():
    hwp.HAction.GetDefault("Goto", hwp.HParameterSet.HGotoE.HSet)
    hwp.HParameterSet.HGotoE.HSet.SetItem("DialogResult", 34)
    hwp.HParameterSet.HGotoE.SetSelectionIndex = 5
    return hwp.HAction.Execute("Goto", hwp.HParameterSet.HGotoE.HSet)
    
    

def go_to_faster_word(word1, word2): 
    hwp.SetMessageBoxMode(0x20000) # 예/아니오 중에 아니오를 누름
    start_pos = hwp.GetPos()

    result1 = search_text(word1)
    hwp.Cancel()
    pos1 = hwp.GetPos() if result1 else None
        
    hwp.SetPos(*start_pos)
    
    result2 = search_text(word2)
    hwp.Cancel()
    pos2 = hwp.GetPos() if result2 else None
    
    hwp.SetPos(*start_pos)
    
    hwp.SetMessageBoxMode(0xF0000) # 경고창이 기본모드로 바뀜
    
    if not result1 and not result2:
        return False
    else:
        if not result1:  # word1을 못 찾은 경우
            hwp.SetPos(*pos2)
            return True
        elif not result2:  # word2를 못 찾은 경우
            hwp.SetPos(*pos1)
            return True
        else:  # 둘 다 찾은 경우
            if pos1[1] < pos2[1]:
                hwp.SetPos(*pos1)
            else:
                hwp.SetPos(*pos2)
            return True



In [ ]:
### 각종 데이터 변환 함수

# 원문자 -> 숫자 변환 함수
def convert_circled_number(text):
    """
    인자로 받은 텍스트의 원문자들을 인식해 일반 숫자 리스트로 변환
    
    Args:
        text (str): 원문자가 포함된 텍스트
        
    Returns:
        list: 변환된 숫자들의 리스트. 원문자가 없으면 빈 리스트 반환
    """
    # 다양한 원문자 매핑
    circled_num_map = {
        '①': 1, '❶': 1, '➀': 1,  # 1에 해당하는 다양한 원문자
        '②': 2, '❷': 2, '➁': 2,  # 2에 해당하는 다양한 원문자
        '③': 3, '❸': 3, '➂': 3,  # 3에 해당하는 다양한 원문자
        '④': 4, '❹': 4, '➃': 4,  # 4에 해당하는 다양한 원문자
        '⑤': 5, '❺': 5, '➄': 5,  # 5에 해당하는 다양한 원문자
        # 필요한 만큼 추가
    }
    
    result = []
    
    # 텍스트를 순회하면서 원문자를 찾아 변환
    for char in text:
        if char in circled_num_map:
            result.append(circled_num_map[char])
            
    return result

def extract_number_from_text(text):
    """
    텍스트에서 숫자(정수 또는 소수)를 추출하는 함수
    
    Args:
        text (str): 숫자가 포함된 텍스트
        
    Returns:
        float: 추출된 숫자. 숫자가 없으면 None 반환
    """
    import re
    
    number_match = re.search(r'(\d+\.?\d*)', text)
    if number_match:
        return float(number_match.group(1))
    return None


import re
def extract_source_text(text):
    pattern = re.compile(r"(고\d+ \d+년 \d+월 \d+번)")
    match = pattern.search(text)
    if match:
        source_text = match.group(1)  # 첫 번째 그룹(괄호 안)에 해당하는 문자열
        return source_text
    return None



In [ ]:
def type_text(text):
    hwp.HAction.GetDefault("InsertText", hwp.HParameterSet.HInsertText.HSet)
    hwp.HParameterSet.HInsertText.Text = text
    hwp.HAction.Execute("InsertText", hwp.HParameterSet.HInsertText.HSet)

In [ ]:
ㅠ

In [ ]:
# 한글에서 조건식을 사용하여 검색하는 법

# Ctrl + Q + M 을 한 뒤 조건식 사용 체크하고 검색
# 

In [ ]:
### 블록 내 정보 조회


# 블록 내 텍스트 포함 여부 (Boolean)
def is_text_in_block(text): 
    hwp.InitScan(range=0xff)  # 0xff <<선택된 범위 내에서 검색
    _, text_blokced = hwp.GetText()  # 텍스트만 추출
    hwp.ReleaseScan() # 스캔을 해제.
    return f"{text}" in text_blokced # 이경우, 해당 target이 text에 포함되어있으면 True, 아니면 False를 반환.

''' any와 next
any는 any( 조건식 for  i in 반복가능객체) 이런구조인거고
next는 next( 반환값 for 반환값(=i) in 반복가능객체 if 조건문) 이렇게되는거 맞아?
'''


# 블록 내 테이블의 갯수 확인
hwp.GetTextFile(format="HWPML2X", option="saveblock:true").count("<TABLE")

    
